<a href="https://colab.research.google.com/github/TheNewLearn/109_2_NTUT_IOS/blob/main/bigdata_hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get -y install openjdk-8-jre-headless
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from google.colab import drive
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
spark = SparkSession.builder.master("local").getOrCreate()
sc = SparkContext.getOrCreate()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  libnss-mdns fonts-dejavu-extra fonts-ipafont-gothic fonts-ipafont-mincho
  fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jre-headless
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 28.2 MB of archives.
After this operation, 104 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jre-headless amd64 8u292-b10-0ubuntu1~18.04 [28.2 MB]
Fetched 28.2 MB in 2s (15.2 MB/s)
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u292-b10-0ubuntu1~18.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u292-b10-0ubuntu1~18.04) ...
Setting up openjdk-8-jre-headless:amd64 (8u292-b10-0ubunt

In [2]:
drive.mount("/content/drive")
path = "/content/drive/My Drive/News_Final.csv"

Mounted at /content/drive


In [36]:
import re
sqlContext = SQLContext(sc)
dataset = sqlContext.read.format("csv").options(header=True, inferSchema='True', quotes="\"",escape="\"").load(path);
dataset.show()



/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+-------+--------------------+--------------------+--------------------+---------+-------------------+-------------------+--------------------+--------+----------+--------+
| IDLink|               Title|            Headline|              Source|    Topic|        PublishDate|     SentimentTitle|   SentimentHeadline|Facebook|GooglePlus|LinkedIn|
+-------+--------------------+--------------------+--------------------+---------+-------------------+-------------------+--------------------+--------+----------+--------+
|99248.0|Obama Lays Wreath...|Obama Lays Wreath...|           USA TODAY|    obama|2002-04-02 00:00:00|                0.0| -0.0533001790889026|      -1|        -1|      -1|
|10423.0|A Look at the Hea...|Tim Haywood, inve...|           Bloomberg|  economy|2008-09-20 00:00:00|  0.208333333333333|  -0.156385810542806|      -1|        -1|      -1|
|18828.0|Nouriel Roubini: ...|Nouriel Roubini, ...|           Bloomberg|  economy|2012-01-28 00:00:00| -0.425210032135381|   0.13975424

In [144]:

news_title = dataset.rdd.map(lambda x:x[1]).flatMap(lambda x:x.split(" ")).map(lambda line: (line,1)).reduceByKey(lambda a,b:a+b)
#sorted(news_title.collect(),key = lambda x: x[1],reverse=True)

news_handline = dataset.rdd.filter(lambda x: type(x[2]) != type(None) ).map(lambda x:x[2]).flatMap(lambda x:x.split(" ")).map(lambda line: (line,1)).reduceByKey(lambda a,b:a+b)
#sorted(news_handline.collect(),key = lambda x: x[1],reverse=True)

news_perday_title = dataset.rdd.map(lambda x:(x[5][0:10],x[1].split(" ")))
#.flatMap(lambda x: x[0])#.map(lambda line: (line,1)).reduceByKey(lambda a,b:a+b)
news_perday_title.collect()
#.map(lambda x:x.replace(",",""))
#news_perday_title.collect()
#news_perday_title.collect()



  



[('2002-04-02',
  ['Obama', 'Lays', 'Wreath', 'at', 'Arlington', 'National', 'Cemetery']),
 ('2008-09-20',
  ['A', 'Look', 'at', 'the', 'Health', 'of', 'the', 'Chinese', 'Economy']),
 ('2012-01-28',
  ['Nouriel', 'Roubini:', 'Global', 'Economy', 'Not', 'Back', 'to', '2008']),
 ('2015-03-01', ['Finland', 'GDP', 'Expands', 'In', 'Q4']),
 ('2015-03-01',
  ['Tourism,',
   'govt',
   'spending',
   'buoys',
   'Thai',
   'economy',
   'in',
   'January']),
 ('2015-03-01',
  ['Intellitec',
   'Solutions',
   'to',
   'Host',
   '13th',
   'Annual',
   'Spring',
   'Microsoft',
   'Dynamics',
   'User',
   'Group']),
 ('2016-02-28', ['Monday,', '29', 'Feb', '2016']),
 ('2015-03-01',
  ['Obama,',
   'stars',
   'pay',
   'a',
   'musical',
   'tribute',
   'to',
   'Ray',
   'Charles']),
 ('2015-03-01',
  ['Fire',
   'claims',
   'more',
   'than',
   '100-year-old',
   'barn',
   'in',
   'Hancock',
   'County']),
 ('2015-03-01',
  ["Microsoft's", 'new', 'Windows', '10', 'ad', 'targets', 'App

In [ ]:
from pyspark.sql.functions import col, lit

def bydaycsv(data,status):
  dataframe = data
  cols = ["IDLink","mean_day_1","mean_day_2"]
  if status == 0:
    dataframe.select(*cols).coalesce(1).write.format('com.databricks.spark.csv').save('Facebook_mean_perday.csv',header = 'true')
  elif status == 1:
    dataframe.select(*cols).coalesce(1).write.format('com.databricks.spark.csv').save('Google_mean_perday.csv',header = 'true')
  elif status == 2:
    dataframe.select(*cols).coalesce(1).write.format('com.databricks.spark.csv').save('Linkedln_mean_perday.csv',header = 'true')

def byhourcsv(data,status):
  cols = ["IDLink"]
  for i in range(0,48):
    cols.append("mean_hour_"+str(i+1))
  dataframe = data
  if status == 0:
    dataframe.select(*cols).coalesce(1).write.format('com.databricks.spark.csv').save('Facebook_mean_hour.csv',header = 'true')
  elif status == 1:
    dataframe.select(*cols).coalesce(1).write.format('com.databricks.spark.csv').save('Google_mean_hour.csv',header = 'true')
  elif status == 2:
    dataframe.select(*cols).coalesce(1).write.format('com.databricks.spark.csv').save('/content/drive/My Drive/inkedln_mean_hour.csv',header = 'true')







dataset = sqlContext.read.format("csv").options(header=True, inferSchema='True', quotes="\"",escape="\"").load(path)
#Facebook
Facebook_pathlist = ["/content/drive/My Drive/Facebook/Facebook_Microsoft.csv","/content/drive/My Drive/Facebook/Facebook_Obama.csv","/content/drive/My Drive/Facebook/Facebook_Palestine.csv"]
fe_dc = sqlContext.read.format("csv").options(header=True, inferSchema='True', quotes="\"",escape="\"").load("/content/drive/My Drive/Facebook/Facebook_Economy.csv")
for i in Facebook_pathlist:
  fe_df = sqlContext.read.format("csv").options(header=True, inferSchema='True', quotes="\"",escape="\"").load(i)
  fe_dc = fe_dc.union(fe_df)


rowMean  = (sum(col(x) for x in fe_dc.columns[1:73]) / 72).alias("mean")
fb_perday = fe_dc.withColumn('mean_day_1',rowMean)
rowMean  = (sum(col(x) for x in fe_dc.columns[73:145]) / 72).alias("mean")
fb_perday = fb_perday.withColumn('mean_day_2',rowMean)


rowMean  = (sum(col(x) for x in fe_dc.columns[1:4]) / 3).alias("mean")
fb_perhour = fe_dc.withColumn('mean_hour_1',rowMean)

timecount = 2
for i in range(4,145,3):
  rowMean  = (sum(col(x) for x in fe_dc.columns[i:i+3]) / 3).alias("mean")
  fb_perhour = fb_perhour.withColumn('mean_hour_'+str(timecount),rowMean)
  timecount+=1







#Google
Google_pathlist = ["/content/drive/My Drive/Google/GooglePlus_Microsoft.csv","/content/drive/My Drive/Google/GooglePlus_Obama.csv","/content/drive/My Drive/Google/GooglePlus_Palestine.csv"]
ge_dc = sqlContext.read.format("csv").options(header=True, inferSchema='True', quotes="\"",escape="\"").load("/content/drive/My Drive/Google/GooglePlus_Economy.csv")
for i in Google_pathlist:
  ge_df = sqlContext.read.format("csv").options(header=True, inferSchema='True', quotes="\"",escape="\"").load(i)
  ge_dc = ge_dc.union(ge_df)





rowMean  = (sum(col(x) for x in ge_dc.columns[1:73]) / 72).alias("mean")
google_perday = ge_dc.withColumn('mean_day_1',rowMean)
rowMean  = (sum(col(x) for x in ge_dc.columns[73:145]) / 72).alias("mean")
google_perday = google_perday.withColumn('mean_day_2',rowMean)
#google_perday.show()

rowMean  = (sum(col(x) for x in ge_dc.columns[1:4]) / 3).alias("mean")
google_perhour = ge_dc.withColumn('mean_hour_1',rowMean)

gtimecount = 2
for i in range(4,145,3):
  rowMean  = (sum(col(x) for x in ge_dc.columns[i:i+3]) / 3).alias("mean")
  google_perhour = google_perhour.withColumn('mean_hour_'+str(gtimecount),rowMean)
  gtimecount+=1

#Linkedln

Linkedln_pathlist = ["/content/drive/My Drive/Linkedln/LinkedIn_Microsoft.csv","/content/drive/My Drive/Linkedln/LinkedIn_Obama.csv","/content/drive/My Drive/Linkedln/LinkedIn_Palestine.csv"]
lk_dc = sqlContext.read.format("csv").options(header=True, inferSchema='True', quotes="\"",escape="\"").load("/content/drive/My Drive/Linkedln/LinkedIn_Economy.csv")
for i in Linkedln_pathlist:
  lk_df = sqlContext.read.format("csv").options(header=True, inferSchema='True', quotes="\"",escape="\"").load(i)
  lk_dc = lk_dc.union(lk_df)




rowMean  = (sum(col(x) for x in lk_dc.columns[1:73]) / 72).alias("mean")
Linkedln_perday = lk_dc.withColumn('mean_day_1',rowMean)
rowMean  = (sum(col(x) for x in lk_dc.columns[73:145]) / 72).alias("mean")
Linkedln_perday = Linkedln_perday.withColumn('mean_day_2',rowMean)
#Linkedln_perday.show()

rowMean  = (sum(col(x) for x in lk_dc.columns[1:4]) / 3).alias("mean")
Linkedln_perhour = lk_dc.withColumn('mean_hour_1',rowMean)
lktimecount = 2
for i in range(4,145,3):
  rowMean  = (sum(col(x) for x in lk_dc.columns[i:i+3]) / 3).alias("mean")
  Linkedln_perhour = Linkedln_perhour.withColumn('mean_hour_'+str(lktimecount),rowMean)
  lktimecount+=1

byhourcsv(Linkedln_perhour,2)
#Linkedln_perhour.show()



  







  




Py4JError: ignored

In [ ]:
def sum_sentiment(dataset,topic,status):
  if status == 0:
    df = dataset.rdd.filter(lambda x:x[4] == topic)
    result = df.map(lambda x:x[6]).reduce(lambda x,y: x+y)
    mean = df.map(lambda x:x[6]).map(lambda x: float(x))
    mean = mean.mean()
    return result, mean
  elif status == 1:
    df = dataset.rdd.filter(lambda x:x[4] == topic)
    result = df.map(lambda x:x[7]).reduce(lambda x,y: x+y)
    mean = df.map(lambda x:x[7]).map(lambda x: float(x))
    mean = mean.mean()
    return result, mean





#obama Sentiment
obama_st = sum_sentiment(dataset,"obama",0)
obama_sh = sum_sentiment(dataset,"obama",1)

#Economy Sentiment
economy_st = sum_sentiment(dataset,"economy",0)
economy_sh = sum_sentiment(dataset,"economy",1)

#microsoft Sentiment
microsoft_st = sum_sentiment(dataset,"microsoft",0)
microsoft_sh = sum_sentiment(dataset,"microsoft",1)

#palestine Sentiment
palestine_st = sum_sentiment(dataset,"palestine",0)
palestine_sh = sum_sentiment(dataset,"palestine",1)

print(obama_st)
print(obama_sh)


print(economy_st)
print(economy_sh)

print(palestine_st)
print(palestine_sh)


print(microsoft_st)
print(microsoft_sh)



(-27.604435365771188, -0.0009648526866749828)
(-507.62019742134794, -0.017742754191588533)
(-350.89723953455365, -0.01034240861632135)
(-1340.1388172155725, -0.03949949355150792)
(-177.61514431068505, -0.020085394584494515)
(-393.38238008336225, -0.04448517246221418)
(51.56650733416493, 0.0023591594534799728)
(-322.2826752054217, -0.014744380785315318)


In [ ]:
#task 4
!pip install pandas
import pandas as pd
def co_occurance_matrix(input_text,top_words,window_size):
    co_occur = pd.DataFrame(index=top_words, columns=top_words)

    for row,nrow in zip(top_words,range(len(top_words))):
        for colm,ncolm in zip(top_words,range(len(top_words))):        
            count = 0
            if row == colm: 
                co_occur.iloc[nrow,ncolm] = count
            else: 
                for single_essay in input_text:
                    essay_split = single_essay.split("")
                    max_len = len(essay_split)
                    top_word_index = [index for index, split in enumerate(essay_split) if row in split]
                    for index in top_word_index:
                        if index == 0:
                            count = count + essay_split[:window_size + 1].count(colm)
                        elif index == (max_len -1): 
                            count = count + essay_split[-(window_size + 1):].count(colm)
                        else:
                            count = count + essay_split[index + 1 : (index + window_size + 1)].count(colm)
                            if index < window_size: 
                                count = count + essay_split[: index].count(colm)
                            else:
                                count = count + essay_split[(index - window_size): index].count(colm)
                co_occur.iloc[nrow,ncolm] = count

    return co_occur
news_per_topic_title_sorted = news_per_topic_title.sortBy(lambda x:x[0][0])
#
obama_tp_title = news_per_topic_title_sorted.filter(lambda x: x[0][0] == "obama")
top_100 = obama_tp_title.top(100,lambda x: x[1])

print(top_100)

#news_title = news_title.collect()

co_matrix = []
'''
top_100 = spark.sparkContext.parallelize(top_100)

top_100 = top_100.map(lambda x:(x[0][1], 1)).reduceByKey(lambda x,y : x+y)

test = dataset.rdd.filter(lambda x: "Obama Lays Wreath at Arlington National Cemetery" in x[1])
test = test.map(lambda x:x[1])
test.collect()
'''



#result = co_occurance_matrix(news_title.collect(),top_100,2)
#result




[(('obama', 'Obama honors Connecticut woman as Teacher of the Year'), 17), (('obama', 'Obama praises Nancy Reagan as a source of comfort, strength'), 13), (('obama', 'Obama says North Carolina law should be overturned'), 12), (('obama', 'President Obama Announces More Key Administration Posts'), 11), (('obama', 'Obama vetoes bill to repeal signature health care law'), 11), (('obama', 'Obama signs bill to deter theft of trade secrets'), 11), (('obama', 'Obama says military hitting IS group harder than ever'), 10), (('obama', 'Obama forced again to rethink troop numbers in Afghanistan'), 10), (('obama', 'Obama signs bill extending privacy protections to allies'), 10), (('obama', 'Obama: Wall Street reforms have worked'), 10), (('obama', "Obama: 'Absolutist' view won't solve encryption debate"), 10), (('obama', "Obama: 'I've made my decision' on Supreme Court nominee"), 10), (('obama', 'Obama says Democrats are good for American government'), 10), (('obama', 'Obama, Macklemore make pitch 

'\ntop_100 = spark.sparkContext.parallelize(top_100)\n\ntop_100 = top_100.map(lambda x:(x[0][1], 1)).reduceByKey(lambda x,y : x+y)\n\ntest = dataset.rdd.filter(lambda x: "Obama Lays Wreath at Arlington National Cemetery" in x[1])\ntest = test.map(lambda x:x[1])\ntest.collect()\n'